# Custom K-Nearest Neighbours
Classifies patients comparing them to others

In [1]:
import pandas as pd 
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from alive_progress import alive_bar
from IPython.display import clear_output


import threading, queue, time

%matplotlib widget

import common
acc = common.loadFile("Accuracy.old")

In [2]:
plt.figure(figsize = [18, 10])
sns.scatterplot(data=acc).set(title="Accuracy Plot", xlabel="Missing Columns", ylabel="Accuracy")
plt.tight_layout(pad=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
cleanData = common.loadFile("CleanedData")
allData = common.loadFile("JoinedData").drop(['LDELTOTAL', 'DIGITSCOR'], axis=1)

for i, j in zip(cleanData.RID, cleanData.VISCODE):
    allData = allData[(allData.RID != i) & (allData.VISCODE != j)]
    
cleanData = cleanData.drop(['RID', 'VISCODE'], axis=1)
allData = allData[(allData.DX.notna()) & (allData.DX != '')]
allData = allData.drop([x for x in list(allData.columns) if x not in list(cleanData.columns)], axis=1) # Drops fields that don't exist in cleanData
allData.replace('', np.NaN, inplace=True)

print(f"CleanData: {len(cleanData)}, allData: {len(allData)}")

CleanData: 1710, allData: 1983


In [4]:
allData = common.loadFile("JoinedData") 
cleanData = common.loadFile("CleanedData").drop(['RID', 'VISCODE', 'PTEDUCAT', 'PTGENDER', 'AGE'], axis=1) # Droping unused fields

allData = allData.drop([x for x in list(allData) if x not in list(cleanData)], axis=1)  # Drops fields that don't exist in cleanData

# Remove rows without diagnosis (This is the field we use to validate)
allData = allData[(allData.DX.notna()) & (allData.DX != '')]
allData = pd.merge(allData, cleanData, how="left", indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)  # Removing cleanData rows from allData
allData.replace('', np.NaN, inplace=True)  # Remove empty string values with np.NaN to make it easier to detect later

allData = allData.astype("float")
cleanData = cleanData.astype("float")

print(f"CleanData: {len(cleanData)}, allData: {len(allData)}")

CleanData: 1710, allData: 5025
